In [46]:
import openpyxl
import openpyxl.styles
import openpyxl.utils

import openpyxl.worksheet.formula
import os
import re

In [47]:
def change_formula(formula: str | None):
    if formula is None:
        return None
    if type(formula) == int or type(formula) == float:
        return formula
    if isinstance(formula, openpyxl.worksheet.formula.ArrayFormula):
        formula = formula.text
    cells = {
        "$C$3": "$D$6",
        "$C$4": "$D$7",
        "$C$5": "$D$8",
        "$C$6": "$D$9",
        "C3": "$D$6",
        "C4": "$D$7",
        "C5": "$D$8",
        "C6": "$D$9",
        "_xlfn.ANCHORARRAY(E3)": "_xlfn.ANCHORARRAY(G6)",
        "E3": "_xlfn.ANCHORARRAY(G6)",
        "M4": "$C$30",
        "M5": "$C$31",
        "M6": "$C$32",
        "M7": "$C$33",
        "M12": "$D$32",
        "M13": "$D$33",
        "L4": "B30",
        "L5": "B31",
        "L6": "B32",
        "L7": "B33",
        "J3": "$D$13",
        "J4": "$D$14",
        "J5": "$D$15",
        "J6": "$D$16",
        "J7": "$D$17",
        "J8": "$D$18",
    }
    r = formula
    for cell in cells.keys():
        r = r.replace(cell, cells[cell])
    return r

def snake_to_pascal(snake_str):
    components = snake_str.split("_")
    pascal_str = "".join(x.title() for x in components)
    return pascal_str

def obtener_maximo_valor_columna(workbook, num_hoja, letra_columna):
    sheet = workbook[workbook.sheetnames[num_hoja]]
    return max((c.row for c in sheet[letra_columna] if c.value is not None))

In [48]:
def integration_formula_replace(formula: str):
    replacements = {
        "_xlfn.NORM.S.DIST": "DISTR.NORM.ESTAND.N",
        "_xlfn.BETA.DIST": "DISTR.BETA.N",
        "_xlfn.GAMMA": "GAMMA",
        "SQRT": "RAIZ",
        "_xlfn.LAMBDA": "LAMBDA",
        "_xlfn.MAP": "MAP",
        "_xlfn.T.DIST": "DISTR.T.N",
        "TRUE": "VERDADERO",
        "FALSE": "FALSO",
        ",": ";",
        "=": "",
        "_xlfn.ANCHORARRAY(G6)": "x",
        "_xlpm.x": "x",
        "0.5": "0,5",
        "1.5": "1,5",
    }
    regex = re.compile("(%s)" % "|".join(map(re.escape, replacements.keys())))
    return regex.sub(lambda mo: replacements[mo.string[mo.start():mo.end()]], formula) 


In [49]:
# =SUMA(MAP(SECUENCIA(MAX($H$6#)-MIN($H$6#)+1;1;MIN($H$6#));LAMBDA(x;(x^B30)*(DISTR.BINOM.N(x;$D$6;$D$7;FALSO)))))

In [50]:
def generate_file_distribution(file_name: str):
    distribution_name = file_name.split(".")[0]

    workbook_template = openpyxl.load_workbook(f"../discrete/{file_name}")
    sheet_0_template = workbook_template[workbook_template.sheetnames[0]]
    sheet_1_template = workbook_template[workbook_template.sheetnames[1]]
    sheet_2_template = workbook_template[workbook_template.sheetnames[2]]

    ppf_formula = sheet_0_template.cell(row=3, column=6).value
    pdf_formula = sheet_1_template.cell(row=3, column=6).value.text
    cdf_formula = sheet_2_template.cell(row=3, column=6).value.text

    mean_formula = sheet_0_template.cell(row=3, column=10).value
    variance_formula = sheet_0_template.cell(row=4, column=10).value
    standard_deviation_formula = sheet_0_template.cell(row=5, column=10).value
    skewness_formula = sheet_0_template.cell(row=6, column=10).value
    kurtosis_formula = sheet_0_template.cell(row=7, column=10).value
    median_formula = sheet_0_template.cell(row=8, column=10).value
    mode_formula = sheet_0_template.cell(row=9, column=10).value

    nc_µ1_formula = sheet_0_template.cell(row=4, column=13).value
    nc_µ2_formula = sheet_0_template.cell(row=5, column=13).value
    nc_µ3_formula = sheet_0_template.cell(row=6, column=13).value
    nc_µ4_formula = sheet_0_template.cell(row=7, column=13).value

    if nc_µ1_formula is None:
        workbook_file = openpyxl.load_workbook("./template_no_moments.xlsx")
    else:
        workbook_file = openpyxl.load_workbook("./template_moments.xlsx")

    sheet_0_file = workbook_file[workbook_file.sheetnames[0]]
    sheet_1_file = workbook_file[workbook_file.sheetnames[1]]
    sheet_2_file = workbook_file[workbook_file.sheetnames[2]]
    sheet_3_file = workbook_file[workbook_file.sheetnames[3]]

    n_params = obtener_maximo_valor_columna(workbook_template, 0, "B") - 2
    for row in range(3, 3 + n_params):
        sheet_0_file.cell(row=row + 3, column=2).value = sheet_0_template.cell(row=row, column=2).value.lower()
        if sheet_0_template.cell(row=row, column=2).value.lower() == "loc":
            sheet_0_file.cell(row=row + 3, column=3).value = "R"
        else:
            sheet_0_file.cell(row=row + 3, column=3).value = "R+"
        sheet_0_file.cell(row=row + 3, column=4).value = sheet_0_template.cell(row=row, column=3).value

        sheet_0_file.cell(row=row + 3, column=2).font = openpyxl.styles.Font(size=10)
        sheet_0_file.cell(row=row + 3, column=2).alignment = openpyxl.styles.Alignment(horizontal="center")
        sheet_0_file.cell(row=row + 3, column=3).font = openpyxl.styles.Font(size=10, italic=True)
        sheet_0_file.cell(row=row + 3, column=3).alignment = openpyxl.styles.Alignment(horizontal="center")
        sheet_0_file.cell(row=row + 3, column=4).font = openpyxl.styles.Font(size=10)
        sheet_0_file.cell(row=row + 3, column=4).alignment = openpyxl.styles.Alignment(horizontal="center")

    parameters = [sheet_0_template.cell(row=row, column=2).value.lower() for row in range(3, 3 + n_params)]

    sheet_0_file["H6"].value = change_formula(ppf_formula)
    sheet_1_file["H6"].value = change_formula(pdf_formula)
    sheet_2_file["H6"].value = change_formula(cdf_formula)
    sheet_3_file["I6"].value = change_formula(ppf_formula).replace("_xlfn.ANCHORARRAY(G6)", "_xlfn.ANCHORARRAY(H6)")
    sheet_0_file["D13"].value = change_formula(mean_formula)
    sheet_0_file["D14"].value = change_formula(variance_formula)
    sheet_0_file["D15"].value = change_formula(standard_deviation_formula)
    sheet_0_file["D16"].value = change_formula(skewness_formula)
    sheet_0_file["D17"].value = change_formula(kurtosis_formula)
    sheet_0_file["D18"].value = change_formula(median_formula)
    sheet_0_file["D19"].value = change_formula(mode_formula)

    if nc_µ1_formula is None:
        sheet_0_file["C30"].value = f"'=SUMA(MAP(SECUENCIA(MAX($H$6#)-MIN($H$6#)+1;1;MIN($H$6#));LAMBDA(x;(x^B{30+n_params-4})*({integration_formula_replace(change_formula(pdf_formula))}))))"
        sheet_0_file["C31"].value = f"'=SUMA(MAP(SECUENCIA(MAX($H$6#)-MIN($H$6#)+1;1;MIN($H$6#));LAMBDA(x;(x^B{31+n_params-4})*({integration_formula_replace(change_formula(pdf_formula))}))))"
        sheet_0_file["C32"].value = f"'=SUMA(MAP(SECUENCIA(MAX($H$6#)-MIN($H$6#)+1;1;MIN($H$6#));LAMBDA(x;(x^B{32+n_params-4})*({integration_formula_replace(change_formula(pdf_formula))}))))"
        sheet_0_file["C33"].value = f"'=SUMA(MAP(SECUENCIA(MAX($H$6#)-MIN($H$6#)+1;1;MIN($H$6#));LAMBDA(x;(x^B{33+n_params-4})*({integration_formula_replace(change_formula(pdf_formula))}))))"
    else:
        sheet_0_file["C30"].value = change_formula(nc_µ1_formula)
        sheet_0_file["C31"].value = change_formula(nc_µ2_formula)
        sheet_0_file["C32"].value = change_formula(nc_µ3_formula)
        sheet_0_file["C33"].value = change_formula(nc_µ4_formula)

    DISTRIBUTION_NOTATION = f"X~{snake_to_pascal(distribution_name)}({','.join(parameters)})"

    sheet_0_file["H5"].value = DISTRIBUTION_NOTATION
    sheet_1_file["G5"].value = DISTRIBUTION_NOTATION
    sheet_2_file["G5"].value = DISTRIBUTION_NOTATION
    sheet_3_file["G5"].value = DISTRIBUTION_NOTATION

    sheet_0_file["F4"].value = sheet_0_file["F4"].value.replace("[D]", distribution_name.upper())
    sheet_1_file["F4"].value = sheet_0_file["F4"].value.replace("[D]", distribution_name.upper())
    sheet_2_file["F4"].value = sheet_0_file["F4"].value.replace("[D]", distribution_name.upper())
    sheet_3_file["F4"].value = sheet_0_file["F4"].value.replace("[D]", distribution_name.upper())

    sheet_1_file["B6"].value = f"='Random Generation'!B6:D{6+len(parameters)-1}"
    sheet_2_file["B6"].value = f"='Random Generation'!B6:D{6+len(parameters)-1}"
    sheet_3_file["B6"].value = f"='Random Generation'!B6:D{6+len(parameters)-1}"

    sheet_1_file["B6"].alignment = openpyxl.styles.Alignment(horizontal="center")
    sheet_1_file["B6"].font = openpyxl.styles.Font(size=10)
    sheet_2_file["B6"].alignment = openpyxl.styles.Alignment(horizontal="center")
    sheet_2_file["B6"].font = openpyxl.styles.Font(size=10)
    sheet_3_file["B6"].alignment = openpyxl.styles.Alignment(horizontal="center")
    sheet_3_file["B6"].font = openpyxl.styles.Font(size=10)

    sheet_3_file["H6"].value = "=(_xlfn.ANCHORARRAY(F6)-1/2)/CONTAR(_xlfn.ANCHORARRAY(F6))"

    workbook_file.save(f"./outputs/{file_name.lower()}")


In [51]:
# for file_name in os.listdir("./excel-files"):
#     if file_name not in os.listdir("./outputs"):
#         try:
#             generate_file_distribution(file_name)
#         except:
#             print(file_name)

In [52]:
# for file_name in os.listdir("./files"):
#     os.remove(f"./files/{file_name}")

In [53]:
for file_name in os.listdir("./outputs"):
    os.remove(f"./outputs/{file_name}")

In [54]:
for file_name in os.listdir("../discrete"):
    try:
        generate_file_distribution(file_name)
        print(file_name)
    except:
        print("xxx" + file_name)

bernoulli.xlsx
binomial.xlsx
geometric.xlsx
hypergeometric.xlsx
logarithmic.xlsx
negative_binomial.xlsx
poisson.xlsx
uniform.xlsx
